# ⚡ Pinokio Lightning.ai - FUSE Fix

**AppImage extraction fix for Lightning.ai**

Extracts AppImage contents to bypass FUSE requirement.

---

In [ ]:
#@title ⚡ Lightning.ai FUSE-Free Setup
#@markdown Extracts AppImage to avoid FUSE dependency

import os
import subprocess
import requests
import time
import json
from pathlib import Path

class PinokioLightningFUSE:
    def __init__(self):
        self.pinokio_path = os.path.expanduser("~/pinokio")
        self.appimage_path = None
        self.extracted_path = None
        self.binary_path = None
        self.server_process = None
        self.tunnel_process = None
        self.tunnel_url = None
        
    def setup_directory(self):
        print("📁 Setting up directory...")
        Path(self.pinokio_path).mkdir(parents=True, exist_ok=True)
        os.chdir(self.pinokio_path)
        print(f"✅ Created directory: {self.pinokio_path}")
        
    def download_and_extract_pinokio(self):
        print("📥 Downloading Pinokio AppImage...")
        self.appimage_path = os.path.join(self.pinokio_path, "Pinokio-3.9.0.AppImage")
        self.extracted_path = os.path.join(self.pinokio_path, "pinokio-extracted")
        
        # Download if not exists
        if not os.path.exists(self.appimage_path):
            try:
                download_url = "https://github.com/pinokiocomputer/pinokio/releases/download/3.9.0/Pinokio-3.9.0.AppImage"
                result = subprocess.run(['wget', '-q', '--show-progress', '-O', self.appimage_path, download_url], capture_output=True, text=True)
                
                if result.returncode != 0:
                    print(f"❌ Download failed: {result.stderr}")
                    return False
                    
                os.chmod(self.appimage_path, 0o755)
                print("✅ Pinokio AppImage downloaded")
            except Exception as e:
                print(f"❌ Download error: {e}")
                return False
        else:
            print("✅ Pinokio AppImage already exists")
        
        # Extract AppImage to bypass FUSE
        if not os.path.exists(self.extracted_path):
            print("🔧 Extracting AppImage (FUSE bypass)...")
            try:
                # Extract AppImage contents
                result = subprocess.run([self.appimage_path, '--appimage-extract'], 
                                      cwd=self.pinokio_path, 
                                      capture_output=True, text=True)
                
                if result.returncode != 0:
                    print(f"❌ Extraction failed: {result.stderr}")
                    return False
                
                # Rename squashfs-root to pinokio-extracted
                squashfs_path = os.path.join(self.pinokio_path, "squashfs-root")
                if os.path.exists(squashfs_path):
                    os.rename(squashfs_path, self.extracted_path)
                    print("✅ AppImage extracted successfully")
                else:
                    print("❌ Extraction directory not found")
                    return False
                    
            except Exception as e:
                print(f"❌ Extraction error: {e}")
                return False
        else:
            print("✅ AppImage already extracted")
        
        # Find the actual binary
        self.binary_path = os.path.join(self.extracted_path, "pinokio")
        if not os.path.exists(self.binary_path):
            # Try alternative paths
            alt_paths = ["AppRun", "usr/bin/pinokio", "bin/pinokio"]
            for alt in alt_paths:
                alt_path = os.path.join(self.extracted_path, alt)
                if os.path.exists(alt_path):
                    self.binary_path = alt_path
                    break
        
        if os.path.exists(self.binary_path) and os.access(self.binary_path, os.X_OK):
            print(f"✅ Found extracted binary: {os.path.basename(self.binary_path)}")
            return True
        else:
            print("❌ Extracted binary not found or not executable")
            return False
    
    def start_pinokio_server(self):
        print("⚡ Starting extracted Pinokio on Lightning.ai...")
        
        if not self.binary_path or not os.path.exists(self.binary_path):
            print("❌ Extracted binary not found")
            return False
            
        try:
            # Lightning.ai compatible environment
            env = os.environ.copy()
            env['PINOKIO_APP_PORT'] = '42000'
            
            # Use extracted directory as app directory
            home = os.path.expanduser("~")
            env['XDG_RUNTIME_DIR'] = os.path.join(home, '.xdg')
            env['XDG_DATA_HOME'] = os.path.join(home, '.local', 'share')
            env['XDG_CONFIG_HOME'] = os.path.join(home, '.config')
            env['XDG_CACHE_HOME'] = os.path.join(home, '.cache')
            env['LD_LIBRARY_PATH'] = os.path.join(self.extracted_path, 'usr', 'lib') + ':' + env.get('LD_LIBRARY_PATH', '')
            
            # Create user directories
            for dir_path in [env['XDG_RUNTIME_DIR'], env['XDG_DATA_HOME'], env['XDG_CONFIG_HOME'], env['XDG_CACHE_HOME']]:
                Path(dir_path).mkdir(parents=True, exist_ok=True)
            
            # Disable D-Bus completely
            env['DBUS_SESSION_BUS_ADDRESS'] = ''
            env['NO_DBUS'] = '1'
            
            print("🚀 Starting extracted server...")
            self.server_process = subprocess.Popen([
                self.binary_path,
                '--no-sandbox',
                '--headless',
                '--disable-dev-shm-usage',
                '--disable-gpu',
                '--disable-software-rasterizer',
                '--no-first-run',
                '--disable-default-apps',
                '--disable-extensions',
                '--disable-plugins',
                '--disable-sync',
                '--disable-background-timer-throttling',
                '--disable-renderer-backgrounding',
                '--disable-backgrounding-occluded-windows',
                '--disable-ipc-flooding-protection',
                '--disable-features=VizDisplayCompositor,TranslateUI',
                '--disable-logging',
                '--disable-background-mode',
                '--user-data-dir=' + os.path.join(home, '.pinokio-data')
            ], env=env, cwd=self.extracted_path, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            # Wait for server
            print("⏳ Waiting for extracted binary startup...")
            for i in range(60):  # Extended timeout for extraction
                try:
                    response = requests.get('http://localhost:42000', timeout=3)
                    if response.status_code == 200:
                        print("✅ ⚡ FUSE-FREE SUCCESS! Server running on port 42000!")
                        return True
                except requests.exceptions.RequestException:
                    time.sleep(1)
                    if i % 15 == 0 and i > 0:
                        print(f"⏳ Still starting extracted binary... ({i+1}/60)")
                        
            print("❌ Server startup timeout - checking process...")
            if self.server_process.poll() is not None:
                stdout, stderr = self.server_process.communicate()
                print(f"❌ Process exited with code {self.server_process.returncode}")
                print(f"Error: {stderr[:500]}")
            return False
            
        except Exception as e:
            print(f"❌ Extracted startup error: {e}")
            return False
    
    def setup_tunnel(self):
        print("🌐 Setting up tunnel...")
        
        try:
            cf_path = os.path.join(self.pinokio_path, 'cloudflared')
            if not os.path.exists(cf_path):
                print("📥 Downloading Cloudflare tunnel...")
                subprocess.run(['wget', '-q', '-O', cf_path, 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64'])
                os.chmod(cf_path, 0o755)
            
            self.tunnel_process = subprocess.Popen([cf_path, 'tunnel', '--url', 'http://localhost:42000'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            print("⏳ Starting tunnel...")
            for _ in range(30):
                line = self.tunnel_process.stdout.readline()
                if 'trycloudflare.com' in line:
                    for part in line.split():
                        if 'trycloudflare.com' in part:
                            self.tunnel_url = part.strip()
                            print(f"✅ ⚡ Tunnel: {self.tunnel_url}")
                            return self.tunnel_url
                time.sleep(1)
            
            return None
            
        except Exception as e:
            print(f"❌ Tunnel error: {e}")
            return None
    
    def cleanup(self):
        if self.server_process:
            self.server_process.terminate()
        if self.tunnel_process:
            self.tunnel_process.terminate()

# LIGHTNING.AI FUSE-FREE EXECUTION
try:
    print("⚡ FUSE-FREE PINOKIO ON LIGHTNING.AI")
    print("=" * 50)
    
    pinokio = PinokioLightningFUSE()
    
    pinokio.setup_directory()
    
    if not pinokio.download_and_extract_pinokio():
        raise Exception("Failed to download and extract Pinokio")
    
    if not pinokio.start_pinokio_server():
        raise Exception("Failed to start extracted Pinokio server")
    
    tunnel_url = pinokio.setup_tunnel()
    
    print("\n⚡ LIGHTNING.AI FUSE-FREE SUCCESS! ⚡")
    print("=" * 50)
    
    if tunnel_url:
        print(f"🌍 Public URL: {tunnel_url}")
        print("📱 Click the link above to access Pinokio!")
    else:
        print("🖥️ Local URL: http://localhost:42000")
    
    print("\n📋 FUSE bypass successful - full Pinokio access available!")
    
    _pinokio_instance = pinokio
    _pinokio_url = tunnel_url or "http://localhost:42000"
    
except Exception as e:
    print(f"\n❌ FUSE-free setup failed: {e}")
    print("\n💡 This approach extracts AppImage contents to bypass FUSE")
    print("💡 If still failing, Lightning.ai may have other restrictions")